# Extract features from a penultimate layer in Emotion English DistilRoBERTa-base model

In [ ]:
# install the transformers library
!pip install transformers

# import required packages
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# work with cuda
device = torch.device('cuda')

In [ ]:
# load tokenizer and model
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to(device)

## Preparation of dataset

### **Option 1** list of texts



In [ ]:
# create list of texts
pred_texts = ['I like that', 'That is annoying', 'This is great!', 'Wouldn´t recommend it.']

### **Option 2** texts from csv file

In [ ]:
# run cell and select file for upload
from google.colab import files
files.upload()

In [24]:
# specify your filename

# note: you can right-click on your file and copy-paste the path to it here
file_name = "/content/test.csv" 

 # select the column in your csv that contains the text to be classified
text_column = "text"

# read in csv
df_pred = pd.read_csv(file_name)
pred_texts = df_pred[text_column].dropna().astype('str').tolist()

## Functions for extracting

In [25]:
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

In [ ]:
model.classifier.dense.register_forward_hook(get_features('feats'))

## Extract features from penultimate layer

In [27]:
# placeholders
PREDICTIONS = []
FEATS = []

# placeholder for batch features
features = {}

for idx, inputs in enumerate(pred_texts):
     
    inputs = tokenizer(inputs, return_tensors="pt")
    inputs = inputs.to(device)

    predictions=model(**inputs).logits

    PREDICTIONS.append(predictions.detach().cpu().numpy())
    FEATS.append(features['feats'].cpu().numpy())
       

In [ ]:
# Inspect features

PREDICTIONS = np.concatenate(PREDICTIONS)
FEATS = np.concatenate(FEATS)

print('preds shape:', PREDICTIONS.shape)
print('feats shape:', FEATS.shape)

In [29]:
# Normalize results

PREDICTIONS = (np.exp(PREDICTIONS)/np.exp(PREDICTIONS).sum(-1,keepdims=True))

FEATS = (np.exp(FEATS)/np.exp(FEATS).sum(-1,keepdims=True))
